In [36]:
import selenium
from selenium import webdriver
from PIL import Image
import io
import os
import time
import requests
import hashlib

In [37]:
DRIVER_PATH = 'C:\Program Files (x86)\Google\Chrome\Application\chromedriver'
wd = webdriver.Chrome(executable_path= DRIVER_PATH)

<ipython-input-37-170965d5c59b>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path= DRIVER_PATH)


In [38]:
wd.get('https://google.com')

In [40]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Facades')


<ipython-input-40-befdfa5cff58>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = wd.find_element_by_css_selector('input.gLFyf')


In [41]:
wd.quit()

In [42]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [43]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [44]:
def search_and_download(search_term:str,driver_path:str,target_path='./images/',number_images=100):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.1)
        
    for elem in res:
        persist_image(target_folder,elem)

In [45]:
search_term = 'Facades'
search_and_download(
search_term = search_term,
    driver_path = DRIVER_PATH
)

<ipython-input-44-7ab28c06d935>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
<ipython-input-42-6fae6806c786>:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


<ipython-input-42-6fae6806c786>:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 101 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQhQtzLBUKhUJCBky81Fj4cfIbKShk4QMgtdg&usqp=CAU - as ./images/facades\4588e3632f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTLQZahZD4Tv-6tjQ5MCINugJJyGW4aGfd4eg&usqp=CAU - as ./images/facades\42ce604e43.jpg
SUCCESS - saved https://cdnassets.hw.net/86/c4/218385bd469bbd685b89b3b2f003/mediafacade-detail-hero.jpg - as ./images/facades\b8a4930fc3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQcmmywai6M5xPerzcRvMYPTnPDOIWzZmuBmA&usqp=CAU - as ./images/facades\9e871254eb.jpg
SUCCESS - saved https://www.archivibe.com/wp-content/uploads/2021/04/ARCHIVIBE-cover-glass-architecture-Mario-Frusca_4.jpg - as ./images/facades\0567c0b247.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRK7TUEhqGP8bK0jmZ87KcwM3ir12x-ygyXIQ&usqp=CAU - as ./images/facades\d4e6d13392.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images

SUCCESS - saved https://www.neuffer.de/sites/de/files/img/window-types/modern-window-facade.jpg - as ./images/facades\f2c943506d.jpg
SUCCESS - saved https://architizer-prod.imgix.net/media/1452080910305Bagh_Jannat_-_Exterior_01.jpg?fit=max&w=1680&q=60&auto=format&auto=compress&cs=strip - as ./images/facades\3d6036d7fa.jpg
SUCCESS - saved https://www.laminam.com/wp-content/uploads/sites/2/2021/07/slider-external-facades-mobile.jpg - as ./images/facades\97f620b65c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSxlExSdtJNe4kwVhQ65YZC7eqze_hOzj5-6w&usqp=CAU - as ./images/facades\65df459c4c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ3LQrJW46gvjb0smsjrI52vrJ283sAFVspgA&usqp=CAU - as ./images/facades\cfd5f7597f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRiYyXxALcxivOREwhOvZI9gNM4dsfn9mb3xQ&usqp=CAU - as ./images/facades\d6b45caf01.jpg
SUCCESS - saved https://static.dezeen.com/uploads/2016/06/office-block